In [64]:
import base64
from openai import OpenAI
from dotenv import load_dotenv

import google.generativeai as genai
load_dotenv()

True

## Gemini model api

In [65]:
genai.configure()

In [66]:
model = genai.GenerativeModel(model_name="gemini-2.0-flash")

In [67]:
temperature = 0.2
top_k = 40 
top_p = 0.95

# Create the model config
generation_config = {
  "temperature": temperature,
  "top_p": top_p,
  "top_k": top_k,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

In [68]:
def prompt_system():
    prompt = """You are a highly skilled medical educator specializing in creating Objective Structured Clinical Examination (OSCE) case scenarios. 
    Your role is to analyze uploaded medical images and generate **realistic, concise, and clinically relevant case prompts** suitable for medical students.  

    ### Guidelines:
    1. **Analyze the Image**:
    - Identify key anatomical, pathological, or radiological features.
    - Consider common clinical conditions associated with the findings in the image.

    2. **Generate a Case Prompt**:
    - Present only the **chief complaint** in a single, well-structured sentence.
    - Do **not** include a diagnosis, additional history, examination findings, or any hints leading directly to the answer.
    - Ensure the case is challenging yet appropriate for a medical student’s level.

    3. **Maintain OSCE Standards**:
    - The case should mimic a real OSCE station where students must:
        - Take a focused history.
        - Perform a relevant physical examination.
        - Develop a differential diagnosis.
    - Avoid excessive details that simplify the case too much.

    4. **Expected Output**:
    - A single plain-text sentence that serves as the case prompt.
    - Example output: *"A 50-year-old male presents with progressive difficulty swallowing over the past two months."*
    - **Do not return JSON, bullet points, explanations, or formatting—only the final case prompt.**

    Follow these instructions strictly to ensure high-quality OSCE cases that enhance medical education.
""".strip()

    return prompt

In [69]:
def prompt(image):
    prompt_template = f"""You are a medical educator tasked with creating a **clinical case scenario** for an OSCE (Objective Structured Clinical Examination). 
    Your goal is to generate a realistic and detailed case prompt based on the provided {image}.

   ### Instructions:
   1. **Analyze the Medical Image**:
      - Carefully examine the uploaded image and identify key clinical features.
      - Consider common pathologies, presentations, and relevant examination findings associated with the {image}.

   2. **Generate a Realistic Case Prompt**:
      - The case prompt should include **only the presenting complaint** (e.g., "A 45-year-old male presents with chest pain for 2 hours").
      - Do not include additional history, examination findings, or diagnosis in the prompt.
      - Ensure the complaint is specific, clinically relevant, and appropriate based on the image.

   3. **Adhere to OSCE Standards**:
      - The case prompt should mimic a real OSCE station, requiring the student to:
        - Take a focused history.
        - Perform a relevant physical examination.
        - Develop a differential diagnosis.
      - Avoid providing excessive details that would make the case too easy or unrealistic.

   4. **Ensure Educational Value**:
      - The case should challenge the student’s clinical reasoning and knowledge.
      - It should be appropriate for the level of a medical student (e.g., not overly complex or obscure).

   ### Input:
   - **Uploaded Medical Image**: [analyzed content from the image]

   ### Expected Output just text:
   A concise and realistic case prompt based on the image, including only the presenting complaint (e.g., 'A 35-year-old female presents with shortness of breath and fever for 3 days').  
   Generate only the case prompt as a single plain text sentence. Do not return JSON, code blocks, or any formatting.
   
   and also the medical condtion you are thinking of and also what is wrong with the image, which type of image is it
   """.strip()

    return prompt_template

In [38]:
def prompt_open():
    prompt_template = f"""You are a medical educator tasked with creating a **clinical case scenario** for an OSCE (Objective Structured Clinical Examination). 
    Your goal is to generate a realistic and detailed case prompt based on the provided image.

   ### Instructions:
   1. **Analyze the Medical Image**:
      - Carefully examine the uploaded image and identify key clinical features.
      - Consider common pathologies, presentations, and relevant examination findings associated with the image.

   2. **Generate a Realistic Case Prompt**:
      - The case prompt should include **only the presenting complaint** (e.g., "A 45-year-old male presents with chest pain for 2 hours").
      - Do not include additional history, examination findings, or diagnosis in the prompt.
      - Ensure the complaint is specific, clinically relevant, and appropriate based on the image.

   3. **Adhere to OSCE Standards**:
      - The case prompt should mimic a real OSCE station, requiring the student to:
        - Take a focused history.
        - Perform a relevant physical examination.
        - Develop a differential diagnosis.
      - Avoid providing excessive details that would make the case too easy or unrealistic.

   4. **Ensure Educational Value**:
      - The case should challenge the student’s clinical reasoning and knowledge.
      - It should be appropriate for the level of a medical student (e.g., not overly complex or obscure).

   ### Input:
   - **Uploaded Medical Image**: [analyzed content from the image]

   ### Expected Output just text:
   A concise and realistic case prompt based on the image, including only the presenting complaint (e.g., 'A 35-year-old female presents with shortness of breath and fever for 3 days').  
   Generate only the case prompt as a single plain text sentence. Do not return JSON, code blocks, or any formatting.
   
   and also the medical condtion you are thinking of and also what is wrong with the image, which type of image is it
   """.strip()

    return prompt_template

In [70]:
system_prompt = prompt_system()

In [71]:
model = genai.GenerativeModel(model_name="gemini-2.0-flash",
                                         system_instruction=system_prompt,
                                         generation_config=generation_config)

In [ ]:
print(model.generate_content(prompt("normal1.jpg")).text)

This appears to be a normal chest X-ray.

A 22-year-old female presents for a routine pre-employment physical examination.



In [85]:
print(model.generate_content(prompt("normal1.png")).text)

A 25-year-old male presents for a routine physical exam.

The image is a normal chest X-ray. The medical condition is a routine check-up with no apparent abnormalities.



In [84]:
print(model.generate_content(prompt("pneumothorax.png")).text)

Medical Condition: Pneumothorax
Image Findings: The image shows a chest X-ray with a visible pleural line and absence of lung markings in the upper right hemithorax, indicative of a pneumothorax.

A 22-year-old male presents with sudden onset of right-sided chest pain and difficulty breathing.



In [77]:
print(model.generate_content(prompt("pneumonia.png")).text)

Medical Condition: Pneumonia
Image Findings: Consolidation in the right lower lobe, suggestive of pneumonia. The image is a chest X-ray (PA view).

A 62-year-old male presents with a persistent cough and increasing breathlessness over the past week.



In [78]:
print(model.generate_content(prompt("acne.png")).text)

Medical Condition: Acne vulgaris
Image Description: Clinical photograph showing facial acne with comedones, papules, and pustules.

Case Prompt: A 16-year-old male presents with a persistent rash on his face for the past year.



### OpenAI vision model

In [53]:
# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

client = OpenAI()

# Encode the local image
image_base64 = encode_image("pneumothorax.png")

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt_open()}, 
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/png;base64,{image_base64}", 
                    },
                },
            ],
        }
    ],
    max_tokens=300,
)

In [54]:
def generate_response_with_image(image_path, prompt):
    """Encodes a local image and sends it along with a prompt to OpenAI's GPT model."""
    
    def encode_image(image_path):
        """Encodes an image to a base64 string."""
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode("utf-8")

    client = OpenAI()
    image_base64 = encode_image(image_path)

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/png;base64,{image_base64}"},
                    },
                ],
            }
        ],
        max_tokens=300,
    )

    return response.choices[0].message.content


In [59]:
print(generate_response_with_image("pneumothorax.png", prompt_open()))

A 60-year-old male presents with a persistent cough and hemoptysis for the past week. 

**Medical condition:** Possible lung cancer or pulmonary infection (e.g., tuberculosis).

**Image type:** Chest X-ray showing potential pathological findings indicating a mass or lesion.


In [57]:
print(generate_response_with_image("pneumonia.png", prompt_open()))

A 60-year-old female presents with chest pain and shortness of breath that started 2 days ago. 

Medical condition: Possible pneumonia or pulmonary edema, based on the presence of hazy infiltrates in the lung fields.

Image type: Chest X-ray showing abnormal lung opacity indicative of consolidation or fluid accumulation.


In [56]:
print(generate_response_with_image("acne.png", prompt_open()))

A 17-year-old male presents with persistent facial acne and irritation for the past 3 months.

**Medical condition**: Acne vulgaris  
**What is wrong with the image**: The image depicts inflammatory acne with pustules and erythema on the face, characteristic of moderate to severe acne vulgaris.  
**Type of image**: Dermatological clinical photograph.


In [88]:
print(generate_response_with_image("normal1.png", prompt_open()))

A 62-year-old male presents with a persistent cough and shortness of breath for the past week.  

The medical condition associated with this image is likely congestive heart failure, and the image is a chest X-ray showing signs of pulmonary congestion.
